In [1]:
# from IPython.display import Image, display
import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor

# config_list = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={"tags": ["gpt-4"]},  # comment out to get all
# )
# When using a single openai endpoint, you can use the following:
config_list = [{"model": "gpt-4", "api_key": os.getenv("OPENAI_API_KEY")}]

In [2]:
import requests
from io import StringIO
from bs4 import BeautifulSoup

def get_hed_vocab():
    if os.path.exists('HEDLatest_terms'):
        with open('HEDLatest_terms', 'r') as fin:
            return fin.read()
    else:
        # URL of the XML file
        url = "https://raw.githubusercontent.com/hed-standard/hed-schemas/main/standard_schema/hedxml/HEDLatest.xml"
        
        # Send a GET request to the URL
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the XML content
            xml_content = response.text
            soup = BeautifulSoup(xml_content, "lxml")
        
            # Find all nodes and extract their names
            all_nodes = soup.find_all('node')
            node_names = [node.find('name', recursive=False).string for node in all_nodes]
        
            return node_names
        else:
            print(f"Failed to retrieve data from the URL. Status code: {response.status_code}") 

In [3]:
import os
from typing import List

from IPython import get_ipython

from autogen import ConversableAgent
from autogen.coding import CodeBlock, CodeExecutor, CodeExtractor, CodeResult, MarkdownCodeExtractor
from hed import HedString, Sidecar, load_schema_version, TabularInput
from hed.errors import ErrorHandler, get_printable_issue_string
from hed.tools.analysis.annotation_util import strs_to_sidecar, to_strlist
from hed.tools.analysis.event_manager import EventManager
from hed.tools.analysis.hed_tag_manager import HedTagManager
from hed.validator import HedValidator

def validate_hed_string(hed_string: str) -> str:
    schema = load_schema_version('8.2.0')
    check_for_warnings = True
    data = hed_string
    hedObj = HedString(data, schema)
    short_string = hedObj.get_as_form('short_tag')

    # Validate the string
    error_handler = ErrorHandler(check_for_warnings=check_for_warnings)
    # validator = HedValidator(schema, def_dict)
    validator = HedValidator(schema)
    issues = validator.validate(hedObj, allow_placeholders=False, error_handler=error_handler)
    if issues:
        issues = get_printable_issue_string(issues, 'Validation issues').strip('\n')
        return issues
    else:
        return 'TERMINATE'

In [4]:
hed_vocab = ",".join(get_hed_vocab())

assistant = ConversableAgent(
    name="CodeWriter",
    system_message="You are an annotator.\n"
    f"You use the vocabulary given to you to create appropriate annotations: {hed_vocab}.\n"
    "Here are some examples\n"
    "Text description: 'The foreground view consists of a large number of ingestible objects, indicating a high quantity. The background view includes an adult human body, outdoors in a setting that includes furnishings, natural features such as the sky, and man-made objects in an urban environment.'\n"
    "Annotation: '(Foreground-view, ((Item-count, High), Ingestible-object)), (Background-view, ((Human, Body, Agent-trait/Adult), Outdoors, Furnishing, Natural-feature/Sky, Urban, Man-made-object))'\n"
    "Text description: In the foreground view, there is an adult human body, an adult male face turned away from the viewer, and a high number of furnishings. In the background view, there are ingestible objects, furnishings, a room indoors, man-made objects, and an assistive device.\n"
    "Annotation: '(Foreground-view, ((Item-count/1, (Human, Body, Agent-trait/Adult)), (Item-count/1, (Human, Body, (Face, Away-from), Male, Agent-trait/Adult)), ((Item-count, High), Furnishing))), (Background-view, (Ingestible-object, Furnishing, Room, Indoors, Man-made-object, Assistive-device))'\n"
    "Once there is no error returned by the validator, the task is done. Returns 'TERMINATE'.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.getenv("OPENAI_API_KEY")}]},
    is_termination_msg=lambda msg: "TERMINATE" in msg['content'].strip().upper(),
    human_input_mode="NEVER",
)

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
)

# # Register the tool signature with the assistant agent.
assistant.register_for_llm(name="validate_hed_string", description="An effective translator")(validate_hed_string)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="validate_hed_string")(validate_hed_string)

/var/folders/c2/k7vlgt7j1y306mj9yk_51jvr0000gn/T/ipykernel_64629/2538793901.py:20: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(xml_content, "lxml")


<function __main__.validate_hed_string(hed_string: str) -> str>

In [5]:
chat_result = user_proxy.initiate_chat(assistant, message=f"Translate this sentence into HED annotations:\nA visual presentation of an image of a green square")

User (to CodeWriter):

Translate this sentence into HED annotations:
A visual presentation of an image of a green square

--------------------------------------------------------------------------------
CodeWriter (to User):

'(Visual-presentation, (Image, Geometric-object/2D-shape/Square, Visual-attribute/Color/CSS-color/Green-color/Green))'

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User (to CodeWriter):



--------------------------------------------------------------------------------
CodeWriter (to User):

***** Suggested tool call (call_yi67PbPoRgwJ0pl4tiSZ0lWS): validate_hed_string *****
Arguments: 
{
"hed_string": "(Visual-presentation, (Image, Geometric-object/2D-shape/Square, Visual-attribute/Color/CSS-color/Green-color/Green))"
}
************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USI